## 데이터셋, 데이터로더

### 내용
1. 데이터셋
    - PyTorch Dataset 클래스
        - Dataset 클래스는 데이터셋을 추상화하여 데이터 로드 및 전처리를 쉽게 도와줌
        - 커스텀 데이터셋을 작성, 다양한 데이터 형식 지원

In [1]:
import torch
from torch.utils.data import Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

2. DataLoader 사용
    - DataLoader 클래스
        - 데이터셋을 배치로 나누고, 데이터셋을 순회할 수 있는 반복자(iterator)를 제공
         - 배치 크기 설정, 셔플링, 병렬 데이터 로딩 지원

In [3]:
from torch.utils.data import DataLoader

In [4]:
# 예시 데이터
data = torch.randn(100, 10)  # 100개의 샘플, 각 샘플은 10개의 특징
labels = torch.randn(100, 1)  # 100개의 샘플에 대한 타겟 값

# 커스텀 데이터셋 생성
dataset = CustomDataset(data, labels)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader 사용 예시
for batch_data, batch_labels in dataloader:
    print(batch_data, batch_labels)

tensor([[-6.7236e-01,  1.5079e-01,  2.4338e-01,  1.5945e+00, -1.4264e+00,
          1.6795e+00,  2.5600e-01,  1.6554e+00,  8.7691e-01, -8.3200e-01],
        [-2.1521e+00,  3.0135e-01, -4.4207e-01, -4.3380e-01,  3.2482e-01,
          1.4192e+00, -3.3470e-01,  3.9579e-01,  3.6500e-03,  1.7356e+00],
        [ 6.3505e-01,  1.3331e-01,  2.8963e-01, -6.9641e-01,  1.2179e+00,
          1.3879e-01,  5.5822e-01,  7.6346e-01, -1.2561e+00, -2.9447e-02],
        [ 6.1954e-01,  7.2735e-01,  1.1923e+00, -2.6734e-01,  6.0208e-01,
         -1.5154e+00, -8.7676e-01, -1.2481e+00,  1.0802e+00,  1.5599e+00],
        [ 5.7981e-01, -7.2042e-01, -1.4010e+00,  9.1696e-02,  1.7877e-01,
         -8.4479e-01,  3.8883e-01,  1.2480e+00,  1.8561e+00, -6.1381e-02],
        [ 5.7388e-01,  9.9058e-01,  6.1275e-01,  1.4692e-01, -3.0161e-01,
         -2.9692e-01, -9.3492e-01,  1.8301e+00, -5.3247e-01, -7.4200e-02],
        [ 4.9375e-01,  5.7288e-02, -2.0241e-01,  1.1273e-01,  7.4454e-01,
         -7.6652e-01, -2.9547e-0

3. 데이터 전처리
    - 데이터 전처리 및 변환
        - torchvision.transforms 모듈을 사용하여 이미지 데이터 전처리
        - 사용자 정의 변환 작성

In [5]:
from torchvision import transforms

In [6]:
# 이미지 데이터 전처리 예시
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 사용자 정의 변환 예시
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={}, std={})'.format(self.mean, self.std)

transform_with_noise = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    AddGaussianNoise(0.0, 0.1)
])

4. 데이터셋 분할
    - 훈련 데이터셋과 검증 데이터셋으로 분할
    - torch.utils.data.random_split 사용

In [7]:
from torch.utils.data import random_split

In [8]:
# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

5. 실습
    - 커스텀 데이터셋 작성 및 DataLoader를 사용하여 데이터 로드
    - 이미지 데이터 전처리 및 변환 적용
    - 데이터셋을 훈련과 검증 데이터셋으로 분할하고, 각 데이터셋에 대한 DataLoader 작성

In [9]:
# 실습 과제 예시
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [10]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

In [11]:
# 예시 데이터
data = torch.randn(100, 3, 32, 32)  # 100개의 이미지 데이터 (3채널, 32x32)
labels = torch.randint(0, 10, (100,))  # 100개의 레이블 (0~9)

# 데이터 전처리 변환
transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 커스텀 데이터셋 생성
dataset = CustomDataset(data, labels, transform=transform)

# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# DataLoader 사용 예시
for batch_data, batch_labels in train_loader:
    print(batch_data, batch_labels)

tensor([[[[-2.1019e+00, -4.3968e+00,  2.1340e+00,  ...,  1.2955e+00,
           -3.4472e-01, -2.4707e+00],
          [-1.6571e+00, -1.3136e+00, -5.2682e+00,  ..., -1.5477e+00,
           -4.4520e-01, -1.2429e-01],
          [-3.0895e+00,  1.5105e+00,  1.4563e+00,  ...,  2.7795e+00,
           -1.4409e+00, -3.7391e+00],
          ...,
          [ 7.8245e-01,  3.2740e-01, -1.8199e-02,  ..., -4.6915e+00,
            3.5509e+00, -1.5456e+00],
          [-3.3966e+00, -3.0930e+00,  2.8879e+00,  ..., -2.0403e+00,
            2.2461e+00, -1.6022e+00],
          [ 1.2963e+00,  2.6889e-01, -3.4840e+00,  ..., -2.7170e+00,
            1.3765e+00, -1.1941e+00]],

         [[ 9.2436e-01,  2.4850e-01, -1.5118e-02,  ..., -1.2292e+00,
            1.0990e+00,  4.7024e-01],
          [-4.7213e+00, -2.3455e+00, -1.3977e+00,  ..., -5.6776e-02,
           -3.2198e+00, -4.4862e+00],
          [-3.0452e+00,  8.2617e-01, -7.4917e-01,  ..., -1.6908e+00,
           -2.3734e+00, -9.5495e-01],
          ...,
     